<a href="https://colab.research.google.com/github/SinisterXI/CS-351L---AI-Lab-GitHub-Repository_2022428/blob/main/Shameer_CS351L_Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq  # Importing heapq to implement the priority queue for A* algorithm
import random  # Importing random for random choices in maze generation
import math    # Importing math for distance calculations

def create_maze(size):
    # Initialize grid with walls ('W')
    grid = [['W' for _ in range(size)] for _ in range(size)]

    # Recursive backtracking to generate the maze
    def carve_passages_from(x, y):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        random.shuffle(directions)

        for dx, dy in directions:
            nx, ny = x + dx * 2, y + dy * 2
            if 0 <= nx < size and 0 <= ny < size and grid[nx][ny] == 'W':
                grid[x + dx][y + dy] = ' '  # Carve path
                grid[nx][ny] = ' '  # Move to the next cell
                carve_passages_from(nx, ny)

    # Start maze generation from top-left corner (0, 0)
    grid[0][0] = ' '  # Start point
    carve_passages_from(0, 0)

    # Ensure the exit has a path
    grid[size - 1][size - 1] = 'E'
    if grid[size - 2][size - 1] == 'W':
        grid[size - 2][size - 1] = ' '  # Make sure the cell above the exit is open
    if grid[size - 1][size - 2] == 'W':
        grid[size - 1][size - 2] = ' '  # Make sure the cell to the left of the exit is open

    return grid

# Function to check if a position is valid (within bounds and not a wall 'W')
def is_valid_position(grid, x, y):
    size = len(grid)
    return 0 <= x < size and 0 <= y < size and grid[x][y] != 'W'  # Position is valid if it's not a wall

# Heuristic function: Calculates the straight-line (Euclidean) distance between the current node and the goal
def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
def a_star(grid, start, goal):
    size = len(grid)
    open_list = []
    heapq.heappush(open_list, (0, start))
    g_score = {start: 0}
    parent = {start: None}
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    # DEBUG: Print the starting and goal positions
    print(f"Starting A* from {start} to {goal}")

    while open_list:
        _, current = heapq.heappop(open_list)

        # DEBUG: Print the current node being processed
        print(f"Processing node: {current}")

        if current == goal:
            print(f"Reached the goal: {goal}")
            break

        for direction in directions:
            next_x = current[0] + direction[0]
            next_y = current[1] + direction[1]
            next_state = (next_x, next_y)

            if is_valid_position(grid, next_x, next_y):
                tentative_g_score = g_score[current] + 1

                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    g_score[next_state] = tentative_g_score
                    f_score = tentative_g_score + heuristic(next_state, goal)
                    heapq.heappush(open_list, (f_score, next_state))
                    parent[next_state] = current  # Update parent dictionary

    if goal not in parent:
        print("Goal is unreachable.")
        return []  # Return an empty path if the goal is unreachable

    # Reconstruct the path from goal to start
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = parent[current]
    path.reverse()

    return path


# Function to print the grid with lines and the path marked
def print_grid_with_path(grid, path):
    grid_with_path = [row.copy() for row in grid]  # Copy the original grid to preserve it
    for (x, y) in path:
        if grid_with_path[x][y] not in ['S', 'E']:  # Don't overwrite start 'S' and exit 'E'
            grid_with_path[x][y] = '*'  # Mark the path with '*'

    # Print the grid with lines to separate cells clearly
    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))  # Print top border line
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')  # Print row with borders between cells
        print('-' * (len(grid_with_path) * 4 + 1))  # Print horizontal line after each row

# Main function to play the maze escape game
def escape_the_maze():
    # Ask the user for grid size
    size = int(input("Enter the maze size (e.g., 6 for a 6x6 maze): "))

    # Create the maze
    grid = create_maze(size)

    # Print the initial maze
    print("\nInitial Maze:")
    print_grid_with_path(grid, [])  # Empty path initially

    # Start the search using A* algorithm
    print("\nSearching for the exit using A* algorithm...\n")
    path = a_star(grid, (0, 0), (size - 1, size - 1))  # Perform A* search from start to exit

    # Print the final maze with the path marked
    print("\nPath to the Exit:")
    print_grid_with_path(grid, path)  # Show the path to the exit

    # Display number of moves it took to escape
    print(f"\nNumber of moves to escape: {len(path) - 1}")

# Run the game
escape_the_maze()


Enter the maze size (e.g., 6 for a 6x6 maze): 6

Initial Maze:

Grid with Path:
-------------------------
|   |   |   |   |   | W |
-------------------------
| W | W | W | W |   | W |
-------------------------
|   | W |   |   |   | W |
-------------------------
|   | W |   | W | W | W |
-------------------------
|   |   |   |   |   |   |
-------------------------
| W | W | W | W |   | E |
-------------------------

Searching for the exit using A* algorithm...

Starting A* from (0, 0) to (5, 5)
Processing node: (0, 0)
Processing node: (0, 1)
Processing node: (0, 2)
Processing node: (0, 3)
Processing node: (0, 4)
Processing node: (1, 4)
Processing node: (2, 4)
Processing node: (2, 3)
Processing node: (2, 2)
Processing node: (3, 2)
Processing node: (4, 2)
Processing node: (4, 3)
Processing node: (4, 4)
Processing node: (4, 5)
Processing node: (5, 4)
Processing node: (5, 5)
Reached the goal: (5, 5)

Path to the Exit:

Grid with Path:
-------------------------
| * | * | * | * | * | W |
----